In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
!mkdir -p ~/.kaggle
from google.colab import drive
from google.colab import files
# Upload your kaggle.json API key
uploaded = files.upload()

drive.mount('/content/drive')

Saving kaggle.json to kaggle.json
Mounted at /content/drive


In [ ]:
import os
import shutil
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/Machine-learning'


In [ ]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Machine-learning/hmnist_28_28_RGB.csv')
y = data['label']
x = data.drop(columns=['label'])
len(x)

10015

In [ ]:
# Preprocess the data
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler()
x, y = oversample.fit_resample(x, y)
x = np.array(x).reshape(-1, 28, 28, 3)
x = (x - np.mean(x)) / np.std(x)



In [ ]:
# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# Load MobileNetV3Small model without the top layers
base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


4334752/4334752 [==============================] - 0s 0us/step


In [ ]:
# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)


In [ ]:
# Create the model
model = Model(inputs=base_model.input, outputs=predictions)



In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
# Create data generators
train_generator = datagen.flow(X_train, Y_train, subset='training')
val_generator = datagen.flow(X_train, Y_train, subset='validation')



In [ ]:
# Train the model
history = model.fit(train_generator,
                    epochs=30,
                    validation_data=val_generator)

Epoch 1/30
939/939 [==============================] - 108s 95ms/step - loss: nan - accuracy: 0.1433 - val_loss: nan - val_accuracy: 0.1386
Epoch 2/30
939/939 [==============================] - 82s 87ms/step - loss: nan - accuracy: 0.1433 - val_loss: nan - val_accuracy: 0.1386
Epoch 3/30
939/939 [==============================] - 84s 90ms/step - loss: nan - accuracy: 0.1433 - val_loss: nan - val_accuracy: 0.1386
Epoch 4/30
939/939 [==============================] - 84s 89ms/step - loss: nan - accuracy: 0.1433 - val_loss: nan - val_accuracy: 0.1386
Epoch 5/30
939/939 [==============================] - 81s 86ms/step - loss: nan - accuracy: 0.1433 - val_loss: nan - val_accuracy: 0.1386
Epoch 6/30
939/939 [==============================] - 84s 89ms/step - loss: nan - accuracy: 0.1433 - val_loss: nan - val_accuracy: 0.1386
Epoch 7/30
939/939 [==============================] - 80s 85ms/step - loss: nan - accuracy: 0.1433 - val_loss: nan - val_accuracy: 0.1386
Epoch 8/30
939/939 [=============

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Accuracy:", accuracy)

# Plot training history
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

TypeError: ignored

In [1]:
# Generate classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(Y_test, y_pred_classes))

NameError: ignored